# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split("\n")]
    
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()]+[target_vocab_to_int['<EOS>']] for sentence in target_text.split("\n")]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    x = tf.placeholder(tf.int32, [None, None], name="input")
    y = tf.placeholder(tf.int32, [None, None])
    
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    target_sequence_length = tf.placeholder(tf.int32, (None, ), name="target_sequence_length")
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name="max_target_len")
    
    source_sequence_length = tf.placeholder(tf.int32, (None, ), name="source_sequence_length")
    
    
    return x, y, learning_rate, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "/usr/lo

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def make_cell(rnn_size, keep_prob):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop_cell = drop_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob = keep_prob)
    
        return drop_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input,
                                             sequence_length = source_sequence_length,
                                             dtype = tf.float32)

    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
        
        
        # Basic decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer) 
        
        # Perform dynamic decoding using the decoder
    training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                   impute_finished=True, 
                                                                   maximum_iterations=max_summary_length)
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

        # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
        # Perform dynamic decoding using the decoder
    inference_decoder_output, _, _= tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_target_sequence_length)
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input,
                                                       target_sequence_length, max_target_sequence_length,
                                                       output_layer, keep_prob)
    
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, target_vocab_to_int['<GO>'],
                                                        target_vocab_to_int['<EOS>'], max_target_sequence_length,
                                                        target_vocab_size, output_layer, batch_size, keep_prob)
    
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length,
                                 source_vocab_size, enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length,
                                                                       max_target_sentence_length, rnn_size, 
                                                                       num_layers, target_vocab_to_int, 
                                                                       target_vocab_size, batch_size, keep_prob, 
                                                                       dec_embedding_size) 
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 7
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 150
decoding_embedding_size = 150
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})

            batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


            batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

            train_acc = get_accuracy(target_batch, batch_train_logits)

            valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/538 - Train Accuracy: 0.2408, Validation Accuracy: 0.3194, Loss: 5.9407
Epoch   0 Batch    1/538 - Train Accuracy: 0.2316, Validation Accuracy: 0.3159, Loss: 5.3708
Epoch   0 Batch    2/538 - Train Accuracy: 0.2766, Validation Accuracy: 0.3384, Loss: 4.6883
Epoch   0 Batch    3/538 - Train Accuracy: 0.2641, Validation Accuracy: 0.3462, Loss: 4.2341
Epoch   0 Batch    4/538 - Train Accuracy: 0.2760, Validation Accuracy: 0.3469, Loss: 3.8594
Epoch   0 Batch    5/538 - Train Accuracy: 0.2984, Validation Accuracy: 0.3473, Loss: 3.5441
Epoch   0 Batch    6/538 - Train Accuracy: 0.3138, Validation Accuracy: 0.3548, Loss: 3.4506
Epoch   0 Batch    7/538 - Train Accuracy: 0.2988, Validation Accuracy: 0.3633, Loss: 3.4590
Epoch   0 Batch    8/538 - Train Accuracy: 0.3143, Validation Accuracy: 0.3778, Loss: 3.3998
Epoch   0 Batch    9/538 - Train Accuracy: 0.3057, Validation Accuracy: 0.3688, Loss: 3.3110
Epoch   0 Batch   10/538 - Train Accuracy: 0.2973, Validation Accuracy

Epoch   0 Batch   89/538 - Train Accuracy: 0.4561, Validation Accuracy: 0.4970, Loss: 1.4894
Epoch   0 Batch   90/538 - Train Accuracy: 0.4944, Validation Accuracy: 0.5130, Loss: 1.4535
Epoch   0 Batch   91/538 - Train Accuracy: 0.4781, Validation Accuracy: 0.5217, Loss: 1.4826
Epoch   0 Batch   92/538 - Train Accuracy: 0.4645, Validation Accuracy: 0.5094, Loss: 1.4423
Epoch   0 Batch   93/538 - Train Accuracy: 0.4408, Validation Accuracy: 0.5000, Loss: 1.4424
Epoch   0 Batch   94/538 - Train Accuracy: 0.4676, Validation Accuracy: 0.5071, Loss: 1.4433
Epoch   0 Batch   95/538 - Train Accuracy: 0.5208, Validation Accuracy: 0.5098, Loss: 1.2930
Epoch   0 Batch   96/538 - Train Accuracy: 0.5041, Validation Accuracy: 0.5295, Loss: 1.2928
Epoch   0 Batch   97/538 - Train Accuracy: 0.4697, Validation Accuracy: 0.5304, Loss: 1.3626
Epoch   0 Batch   98/538 - Train Accuracy: 0.5033, Validation Accuracy: 0.5259, Loss: 1.2630
Epoch   0 Batch   99/538 - Train Accuracy: 0.4588, Validation Accuracy

Epoch   0 Batch  178/538 - Train Accuracy: 0.5854, Validation Accuracy: 0.5916, Loss: 0.7052
Epoch   0 Batch  179/538 - Train Accuracy: 0.5627, Validation Accuracy: 0.5874, Loss: 0.7453
Epoch   0 Batch  180/538 - Train Accuracy: 0.5954, Validation Accuracy: 0.5847, Loss: 0.7164
Epoch   0 Batch  181/538 - Train Accuracy: 0.5402, Validation Accuracy: 0.5829, Loss: 0.7592
Epoch   0 Batch  182/538 - Train Accuracy: 0.5391, Validation Accuracy: 0.5803, Loss: 0.7476
Epoch   0 Batch  183/538 - Train Accuracy: 0.6008, Validation Accuracy: 0.5950, Loss: 0.6799
Epoch   0 Batch  184/538 - Train Accuracy: 0.5938, Validation Accuracy: 0.5968, Loss: 0.6862
Epoch   0 Batch  185/538 - Train Accuracy: 0.5783, Validation Accuracy: 0.6117, Loss: 0.7075
Epoch   0 Batch  186/538 - Train Accuracy: 0.5880, Validation Accuracy: 0.6195, Loss: 0.7113
Epoch   0 Batch  187/538 - Train Accuracy: 0.6186, Validation Accuracy: 0.6143, Loss: 0.6699
Epoch   0 Batch  188/538 - Train Accuracy: 0.5924, Validation Accuracy

Epoch   0 Batch  267/538 - Train Accuracy: 0.6732, Validation Accuracy: 0.6426, Loss: 0.5234
Epoch   0 Batch  268/538 - Train Accuracy: 0.6760, Validation Accuracy: 0.6525, Loss: 0.4927
Epoch   0 Batch  269/538 - Train Accuracy: 0.6662, Validation Accuracy: 0.6575, Loss: 0.5155
Epoch   0 Batch  270/538 - Train Accuracy: 0.6623, Validation Accuracy: 0.6507, Loss: 0.5239
Epoch   0 Batch  271/538 - Train Accuracy: 0.6453, Validation Accuracy: 0.6454, Loss: 0.5263
Epoch   0 Batch  272/538 - Train Accuracy: 0.6498, Validation Accuracy: 0.6497, Loss: 0.5527
Epoch   0 Batch  273/538 - Train Accuracy: 0.6775, Validation Accuracy: 0.6671, Loss: 0.5263
Epoch   0 Batch  274/538 - Train Accuracy: 0.6279, Validation Accuracy: 0.6669, Loss: 0.5514
Epoch   0 Batch  275/538 - Train Accuracy: 0.6633, Validation Accuracy: 0.6575, Loss: 0.5333
Epoch   0 Batch  276/538 - Train Accuracy: 0.6566, Validation Accuracy: 0.6566, Loss: 0.5217
Epoch   0 Batch  277/538 - Train Accuracy: 0.6596, Validation Accuracy

Epoch   0 Batch  356/538 - Train Accuracy: 0.7383, Validation Accuracy: 0.7047, Loss: 0.3406
Epoch   0 Batch  357/538 - Train Accuracy: 0.7514, Validation Accuracy: 0.7138, Loss: 0.3702
Epoch   0 Batch  358/538 - Train Accuracy: 0.7400, Validation Accuracy: 0.7196, Loss: 0.3695
Epoch   0 Batch  359/538 - Train Accuracy: 0.7346, Validation Accuracy: 0.7129, Loss: 0.3660
Epoch   0 Batch  360/538 - Train Accuracy: 0.7236, Validation Accuracy: 0.7251, Loss: 0.3742
Epoch   0 Batch  361/538 - Train Accuracy: 0.7602, Validation Accuracy: 0.7488, Loss: 0.3597
Epoch   0 Batch  362/538 - Train Accuracy: 0.7593, Validation Accuracy: 0.7493, Loss: 0.3470
Epoch   0 Batch  363/538 - Train Accuracy: 0.7411, Validation Accuracy: 0.7438, Loss: 0.3479
Epoch   0 Batch  364/538 - Train Accuracy: 0.7125, Validation Accuracy: 0.7113, Loss: 0.3840
Epoch   0 Batch  365/538 - Train Accuracy: 0.7254, Validation Accuracy: 0.7136, Loss: 0.3569
Epoch   0 Batch  366/538 - Train Accuracy: 0.7459, Validation Accuracy

...

Epoch   6 Batch  160/538 - Train Accuracy: 0.9788, Validation Accuracy: 0.9750, Loss: 0.0125
Epoch   6 Batch  161/538 - Train Accuracy: 0.9699, Validation Accuracy: 0.9753, Loss: 0.0118
Epoch   6 Batch  162/538 - Train Accuracy: 0.9901, Validation Accuracy: 0.9746, Loss: 0.0113
Epoch   6 Batch  163/538 - Train Accuracy: 0.9859, Validation Accuracy: 0.9746, Loss: 0.0155
Epoch   6 Batch  164/538 - Train Accuracy: 0.9834, Validation Accuracy: 0.9767, Loss: 0.0142
Epoch   6 Batch  165/538 - Train Accuracy: 0.9812, Validation Accuracy: 0.9718, Loss: 0.0102
Epoch   6 Batch  166/538 - Train Accuracy: 0.9850, Validation Accuracy: 0.9751, Loss: 0.0097
Epoch   6 Batch  167/538 - Train Accuracy: 0.9767, Validation Accuracy: 0.9760, Loss: 0.0190
Epoch   6 Batch  168/538 - Train Accuracy: 0.9760, Validation Accuracy: 0.9775, Loss: 0.0163
Epoch   6 Batch  169/538 - Train Accuracy: 0.9938, Validation Accuracy: 0.9775, Loss: 0.0103
Epoch   6 Batch  170/538 - Train Accuracy: 0.9821, Validation Accuracy

Epoch   6 Batch  249/538 - Train Accuracy: 0.9786, Validation Accuracy: 0.9583, Loss: 0.0103
Epoch   6 Batch  250/538 - Train Accuracy: 0.9801, Validation Accuracy: 0.9574, Loss: 0.0127
Epoch   6 Batch  251/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9629, Loss: 0.0134
Epoch   6 Batch  252/538 - Train Accuracy: 0.9849, Validation Accuracy: 0.9709, Loss: 0.0139
Epoch   6 Batch  253/538 - Train Accuracy: 0.9794, Validation Accuracy: 0.9695, Loss: 0.0110
Epoch   6 Batch  254/538 - Train Accuracy: 0.9811, Validation Accuracy: 0.9641, Loss: 0.0171
Epoch   6 Batch  255/538 - Train Accuracy: 0.9904, Validation Accuracy: 0.9632, Loss: 0.0111
Epoch   6 Batch  256/538 - Train Accuracy: 0.9881, Validation Accuracy: 0.9631, Loss: 0.0138
Epoch   6 Batch  257/538 - Train Accuracy: 0.9834, Validation Accuracy: 0.9656, Loss: 0.0119
Epoch   6 Batch  258/538 - Train Accuracy: 0.9717, Validation Accuracy: 0.9677, Loss: 0.0147
Epoch   6 Batch  259/538 - Train Accuracy: 0.9847, Validation Accuracy

Epoch   6 Batch  338/538 - Train Accuracy: 0.9868, Validation Accuracy: 0.9688, Loss: 0.0122
Epoch   6 Batch  339/538 - Train Accuracy: 0.9775, Validation Accuracy: 0.9691, Loss: 0.0122
Epoch   6 Batch  340/538 - Train Accuracy: 0.9785, Validation Accuracy: 0.9645, Loss: 0.0108
Epoch   6 Batch  341/538 - Train Accuracy: 0.9793, Validation Accuracy: 0.9656, Loss: 0.0127
Epoch   6 Batch  342/538 - Train Accuracy: 0.9782, Validation Accuracy: 0.9638, Loss: 0.0128
Epoch   6 Batch  343/538 - Train Accuracy: 0.9922, Validation Accuracy: 0.9695, Loss: 0.0118
Epoch   6 Batch  344/538 - Train Accuracy: 0.9820, Validation Accuracy: 0.9656, Loss: 0.0096
Epoch   6 Batch  345/538 - Train Accuracy: 0.9814, Validation Accuracy: 0.9629, Loss: 0.0126
Epoch   6 Batch  346/538 - Train Accuracy: 0.9721, Validation Accuracy: 0.9648, Loss: 0.0164
Epoch   6 Batch  347/538 - Train Accuracy: 0.9820, Validation Accuracy: 0.9666, Loss: 0.0108
Epoch   6 Batch  348/538 - Train Accuracy: 0.9807, Validation Accuracy

Epoch   6 Batch  427/538 - Train Accuracy: 0.9787, Validation Accuracy: 0.9688, Loss: 0.0113
Epoch   6 Batch  428/538 - Train Accuracy: 0.9896, Validation Accuracy: 0.9664, Loss: 0.0070
Epoch   6 Batch  429/538 - Train Accuracy: 0.9868, Validation Accuracy: 0.9719, Loss: 0.0127
Epoch   6 Batch  430/538 - Train Accuracy: 0.9668, Validation Accuracy: 0.9718, Loss: 0.0154
Epoch   6 Batch  431/538 - Train Accuracy: 0.9738, Validation Accuracy: 0.9705, Loss: 0.0138
Epoch   6 Batch  432/538 - Train Accuracy: 0.9741, Validation Accuracy: 0.9728, Loss: 0.0144
Epoch   6 Batch  433/538 - Train Accuracy: 0.9705, Validation Accuracy: 0.9673, Loss: 0.0261
Epoch   6 Batch  434/538 - Train Accuracy: 0.9777, Validation Accuracy: 0.9574, Loss: 0.0118
Epoch   6 Batch  435/538 - Train Accuracy: 0.9850, Validation Accuracy: 0.9524, Loss: 0.0134
Epoch   6 Batch  436/538 - Train Accuracy: 0.9605, Validation Accuracy: 0.9489, Loss: 0.0159
Epoch   6 Batch  437/538 - Train Accuracy: 0.9834, Validation Accuracy

Epoch   6 Batch  516/538 - Train Accuracy: 0.9779, Validation Accuracy: 0.9581, Loss: 0.0122
Epoch   6 Batch  517/538 - Train Accuracy: 0.9875, Validation Accuracy: 0.9597, Loss: 0.0100
Epoch   6 Batch  518/538 - Train Accuracy: 0.9869, Validation Accuracy: 0.9638, Loss: 0.0136
Epoch   6 Batch  519/538 - Train Accuracy: 0.9816, Validation Accuracy: 0.9622, Loss: 0.0115
Epoch   6 Batch  520/538 - Train Accuracy: 0.9760, Validation Accuracy: 0.9627, Loss: 0.0127
Epoch   6 Batch  521/538 - Train Accuracy: 0.9852, Validation Accuracy: 0.9670, Loss: 0.0116
Epoch   6 Batch  522/538 - Train Accuracy: 0.9816, Validation Accuracy: 0.9684, Loss: 0.0111
Epoch   6 Batch  523/538 - Train Accuracy: 0.9879, Validation Accuracy: 0.9632, Loss: 0.0097
Epoch   6 Batch  524/538 - Train Accuracy: 0.9723, Validation Accuracy: 0.9625, Loss: 0.0091
Epoch   6 Batch  525/538 - Train Accuracy: 0.9829, Validation Accuracy: 0.9625, Loss: 0.0129
Epoch   6 Batch  526/538 - Train Accuracy: 0.9728, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    unknown_id = vocab_to_int['<UNK>']
    sentence_int = []
    for word in sentence.lower().split():
        try:
            next_word_id = vocab_to_int[word]
        except KeyError:
            next_word_id = unknown_id
        sentence_int.append(next_word_id)
    return sentence_int


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [ ]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from checkpoints/dev


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.